<a href="https://colab.research.google.com/github/amirul2k9/3D_detection_MAESTRO/blob/master/Copy_of_Mashary_SnsScrape_Arabizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q snscrape

## Data Extraction

In [ ]:
import os
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import date
import itertools

In [ ]:
today = date.today()
end_date = today
end_date

datetime.date(2023, 1, 22)

In [ ]:
from_date = "2022-11-01"

In [ ]:
os.system(f"snscrape --max-results 150 twitter-search ' i am happy> result-tweets.txt")
if os.stat("result-tweets.txt").st_size == 0:
  counter = 0
else:
  df = pd.read_csv("result-tweets.txt", names=["link"])
  counter = df.size

counter

150

In [ ]:
location = '24.774265,46.738586'
#extracted_tweets="snscrape --format '{content!r}' --max-results 150 --location {location} twitter-search 'يا حقير' > extracted-tweets.txt"
extracted_tweets="snscrape --format '{content!r}' --max-results 150 twitter-search 'يا حقير' > extracted-tweets.txt"
os.system(extracted_tweets)
if os.stat("extracted-tweets.txt").st_size == 0:
  print('No tweets')
else:
  df = pd.read_csv("extracted-tweets.txt", names=["content"])
df

,content
0,'@fahddeepaji1 كان قلت من اول ان المصيبه الكبر...
1,'@SamihTweets @maryadamm اتفوه عليك ياكلب ياتا...
2,'@fathi_bashagha الفجور في الخصومة دليل التفاه...
3,'@antfadt كون ضربك الموت وخلصنه منك يا حقير'
4,'ابلع يا اصلع ابلع يا حقير'
...,...
145,'@anililter تف تف تف تف تف تف ياحقير ...
146,'@hus700 @Eduarddooo55 عليك اللعنه ياطرطور انت...
147,'@JSlDYTLc9udtU8S @Eduarddooo55 عليك اللعنه يا...
148,'@mohammed_alamad اسيادك يادلخ يامزمرماتسوجزمه...


## Pre-processing

In [ ]:
# remove emojis
import re
df['content']=df['content'].str.lower()
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

df['content'] = df['content'].apply(lambda x:emoji_pattern.sub(r'', x))
len(df.index)

150

In [ ]:
#remove tags
def removeTags(tweet):
  sentence = " ".join(filter(lambda x:x[0]!="'", tweet.split()))
  return sentence

df['content'] = df['content'].apply(lambda x:removeTags(x))
len(df.index)

150

In [ ]:
#remove links
def remove_links(tweet):
  sentence = re.sub(r'http\S+', '', tweet)
  return sentence

df['content'] = df['content'].apply(lambda x:remove_links(x))
df
len(df.index)

150

In [ ]:
from pandas.core.arrays import string_
# further preprocessing for Arabic

def remove_links(tweet):
  sentence = re.sub(r'\'+$', '', tweet)
  sentence=re.sub("#[A-Za-z0-9_]+","", sentence)
  sentence=re.sub("u[A-Za-z0-9_]+","", tweet)
  sentence=re.sub("@[A-Za-z0-9_]+","", sentence) # remove username or mention in the tweets
  sentence=re.sub("#","", sentence)
  sentence=re.sub("&;","", sentence)
  sentence=re.sub("#\w\*","", sentence) # hashtags

  return sentence


df['content'] = df['content'].apply(lambda x:remove_links(x))
df = df.replace('&',' ', regex=True)
df = df.replace(';',' ', regex=True)
df=df.replace(r'\\n',  '', regex=True)  # remove new line command (\n)
df=df.replace(r'\\',  '', regex=True)
df = df[df['content'].str.split().str.len().gt(3)]  # delete rows with three or less words
df['content']= df['content'].str.replace(r'(\w)\1{%d,}'%(3-1), r'\1') # skip repeated character
max_size = 20
df['content'] = df['content'].str.split(n=max_size).str[:max_size].str.join(' ') # set limit to number of words in each row
df = df.drop_duplicates()  # drop duplicate rows
df['content']=df['content'].str.strip()
df=df.dropna()
len(df.index)

<ipython-input-139-ed50cb1131d1>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content']= df['content'].str.replace(r'(\w)\1{%d,}'%(3-1), r'\1') # skip repeated character


135

In [ ]:
df

,content
0,كان قلت من اول ان المصيبه الكبرى عندك هي الفلو...
1,اتفوه عليك ياكلب ياتافه ياحقير'
2,الفجور في الخصومة دليل التفاهة و الحقارة تشكر ...
3,كون ضربك الموت وخلصنه منك يا حقير'
4,يا اصلع ابلع يا حقير'
...,...
144,وقتی فقیر میشود، خوبیهایش هم حقیر میشوند؛ اما ...
145,تف تف تف تف تف تف ياحقير الا فوق راسك
146,عليك اللعنه ياطرطور انت اكبر نصاب ياحقير يانذل...
148,اسيادك يادلخ يامزمرماتسوجزمه من جزماتهم ياحقير'


In [ ]:
from pandas.core.arrays import string_
# further preprocessing

def remove_links(tweet):
  sentence = re.sub(r'\'+$', '', tweet)
  sentence=re.sub("#[A-Za-z0-9_]+","", sentence)
  sentence=re.sub("\\u[A-Za-z0-9_]+","", sentence)
  sentence=re.sub("@[A-Za-z0-9_]+","", sentence) # remove username or mention in the tweets
  sentence=re.sub('[^\x00-\x7F]+', '', sentence)
  sentence=re.sub("#","", sentence)
  sentence=re.sub("&;","", sentence)
  sentence=re.sub("#\w\*","", sentence) # hashtags
  sentence=re.sub("(?<![\w-])\w(?![\w-])","", sentence)
  #sentence=re.sub("\s{2,}|\t","", sentence)  # remove blank_spaces
  #sentence=re.sub("(RT|rt|FAV|fav|VIA|via)","", sentence) # twitter_reserved_words
  #sentence=re.sub (r'http\S+', "", sentence) # remove http links
  #sentence=re.sub(r'bit.ly/\S+', "", sentence) # remove bitly links
  #sentence=sentence.strip('[link]') # remove [links]
  #sentence=re.sub('(RT\s@[A-Za-z]+[A-Za-z0--9-*]+)', "", sentence) # remove retweet
  #sentence=re.sub('(@[A-Za-z]+[A-Za-z0--9-_]+)', "", sentence) # remove tweeted at
  sentence=sentence.encode('ascii', 'ignore').decode('ascii')


  return sentence
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords

#stop_words = stopwords.words('english')
#df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df['content'] = df['content'].apply(lambda x:remove_links(x))
df = df.replace('&',' ', regex=True)
df = df.replace(';',' ', regex=True)
df=df.replace(r'\\n',  '', regex=True)  # remove new line command (\n)
df=df.replace(r'\\',  '', regex=True)
#df =  df[df['content'].str.contains('[A-Za-z]')]  # delete the rows having only sybmols, number (not alphabet)
df = df[df['content'].str.split().str.len().gt(3)]  # delete rows with three or less words
df['content']= df['content'].str.replace(r'(\w)\1{%d,}'%(3-1), r'\1') # skip repeated character
#df['content'] = df['content'].str[:100]   # set limit to number of character in each row
df = df.drop_duplicates()  # drop duplicate rows
df['content']=df['content'].str.strip()
df=df.dropna()
df
len(df.index)

error: ignored

In [ ]:
df

,content
0,
1,
2,
3,
4,\
...,...
95,
96,
97,\ \ ...
98,@amr_hebish


In [ ]:
#save the file
df.to_csv('Emotional_English_dataset23_tru_4.csv', index=False)

In [ ]:
#combine into single csv
df1 = pd.read_csv('/content/Emotional_English_dataset23_dis_1.csv', sep='\t')
df2 = pd.read_csv('/content/Emotional_English_dataset23_dis_2.csv', sep='\t')
df3 = pd.read_csv('/content/Emotional_English_dataset23_dis_3.csv', sep='\t')
df4 = pd.read_csv('/content/Emotional_English_dataset23_dis_4.csv', sep='\t')
#df5 = pd.read_csv('/content/Emotional_English_dataset23_sad_5.csv', sep='\t')
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df = df.drop_duplicates()  # drop duplicate rows
df.to_csv('Emotional_English_dataset23_disgust.csv', index=False)
df

,content
0,had to end stream early cause am feeling bit...
1,am genuinely feeling kinda sick
2,am not ready to watch it am feeling sick
3,glad you're feeling better. am still barely f...
4,! havent been posting in while because my bi...
...,...
418,"am frasturated, too. tweets are getting very n..."
419,make naa show me ur mercy
420,am so frasturated irratated !
421,am frasturated of the fact that minds like the...


In [ ]:
df = pd.read_csv("/content/Emotional_English_dataset23_optimism_new.csv")
#df['content'] = df['content'].str[:100]   # set limit to number of character in each row
max_size = 20

df['content'] = df['content'].str.split(n=max_size).str[:max_size].str.join(' ') # set limit to number of words in each row
df.to_csv('Emotional_English_dataset23_optimism.csv', index=False)
df

,content
0,am optimistic. we the people.
1,bought couple of tickets for the madonna' cele...
2,im not sure if you mean it
3,top story: : never been more optimistic about ...
4,those mappings ... we should just shove wkt ar...
...,...
644,am proud of my thickness
645,felt tired but motivated to fly by this weeken...
646,am gona do it all over again tomorrow is just ...
647,monday to my beloved baby! am still here in th...


In [ ]:
#combine into single csv
df11 = pd.read_csv('/content/Emotional_English_dataset23_joy.csv', sep='\t')
df21 = pd.read_csv('/content/Emotional_English_dataset23_angry.csv', sep='\t')
df = pd.concat([df11, df21], ignore_index=True)
df.to_csv('Emotional_English_dataset23_joy_ang.csv', index=False)

NameError: ignored

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Download the file
from google.colab import files
files.download('Emotional_Arabizi_labeled.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# create regex pattern out of the list of words
anger_kw = '|'.join(['bad','anger', 'pissed', 'annoyed', 'angry','fury','fuck you','so angry','fuck off','wrath','outrage','exasperation','irritability','resentment','animosity','animosity','enmity','hostility','grudge','acrimony','bitterness','rage','frustration','exasperation','annoyance','aggravation'])
anticipation_kw = '|'.join(["looking forward", "excited", "can't wait", "coming soon" ,"countdown", "upcoming", "next", "future", "gonna be", "will be", "about to", 'anticipation','i gotta see','anticipated','expected','hope','wish','looking for', 'hoping','wishing', 'being in suspense','preparing for','waiting for'])
disgust_kw = '|'.join(["disgust", "revulsion", "repulsion", "loathing", "abhorrence", "contempt", "scorn", "revile", "vilify", "dislike", 'disgusted','frasturated','feeling sick','turned off by','revolted by','cant stand'])
fear_kw = '|'.join(['fear','i am afraid','i am anxious', 'anxious', 'apprehensive','concerned','dread','frightened','horrified','panicked','terrified', 'violence',"fear", "afraid", "terror", "terrorized", "scared", "frightened", "petrified", "alarmed", "anxious", "panicked"])
joy_kw = '|'.join(['thank you', "happy", "joyful", "content", "pleased", "delighted", "glad", "overjoyed", "thrilled", "ecstatic", "grateful", "content", "blissful"])
love_kw = '|'.join(['love you','passionated', 'passionate','fond','loving','love', "affection", "devotion", "passion", "adoration", "fondness", "warmth", "tenderness", "intimacy", "attachment", "cherish", "enjoy", "like", "caring", "heart"])
optimism_kw = '|'.join(['optimism','i am optimistic','i am hopeful','hopeful','i am positive', 'confident','upbeat','rosy','i am bright', "hope", "optimistic", "positive", "bright", "promising", "encouraging", "upbeat", "motivated", "expectant", "anticipating", "aspiring", "ambitious", "forward-looking", "looking ahead"])
pessimism_kw = '|'.join(['pessimism','i am pessimistic','i am negative', 'despairing','skeptical', "disillusioned", "doubtful", "gloomy", "hopeless", "negative", "pessimistic", "skeptical", "unenthusiastic", "unoptimistic", "worried"])
sadness_kw = '|'.join(['sadness','i am so sorry','am so sorry', 'sorry','i am unhappy','i am sad','despondent','i am gloomy','dejected','downhearted', 'depressed','sad', 'disappointed', "heartbroken", "devastated", "miserable", "unhappy", "down", "grief", "sorrow", "gloomy", "melancholic", "despair", "despondent"])
surprise_kw = '|'.join(['surprise', "unexpected", "unforeseen", "unpredicted", "shocked", "astounded", "amazed", "stunned", "startled", "dumbfounded", "flabbergasted", 'surprise','i am astounded','i am amazed','i am shocked','i am astonished','i am dumbfounded','i am traumatised'])
trust_kw = '|'.join(['trust','loyalty', 'faith', 'reliability', 'credibility', 'honesty', 'integrity', 'authenticity', 'sincerity' , 'dependability', 'confidence', 'trust','i am sure', '100% sure','99% sure','i am assured','i am relying','i am certain','absolutely sure','i am self-assured','i have certainty',' i have belief in you','belief'])
neutral_kw = '|'.join(["normal", "ordinary", "average", "calm", "cool", "collected", "composed", "controlled", "level-headed", "matter-of-fact", "unexcited", "unresponsive"])


#df = pd.DataFrame(data=words, columns=['words'])
df = pd.read_csv("/content/Emotional_English_dataset23_optimism.csv")
df['content'] = df['content'].str[:200]

df['Anger'] = df['content'].str.contains(anger_kw).astype(int)
df['Anticipation'] = df['content'].str.contains(anticipation_kw).astype(int)
df['Disgust'] = df['content'].str.contains(disgust_kw).astype(int)
df['Fear'] = df['content'].str.contains(fear_kw).astype(int)
df['Joy'] = df['content'].str.contains(joy_kw).astype(int)
df['Love'] = df['content'].str.contains(love_kw).astype(int)
df['Optimism'] = df['content'].str.contains(optimism_kw).astype(int)
df['Pessimism'] = df['content'].str.contains(pessimism_kw).astype(int)
df['Sadness'] = df['content'].str.contains(sadness_kw).astype(int)
df['Surprise'] = df['content'].str.contains(surprise_kw).astype(int)
df['Trust'] = df['content'].str.contains(trust_kw).astype(int)
df['Neutral'] = df['content'].str.contains(neutral_kw).astype(int)

print(df)

                                               content  Anger  Anticipation  \
0                        am optimistic. we the people.      0             0   
1    bought couple of tickets for the madonna' cele...      0             0   
2                           im not sure if you mean it      0             0   
3    top story: : never been more optimistic about ...      0             1   
4    those mappings ... we should just shove wkt ar...      0             0   
..                                                 ...    ...           ...   
644                           am proud of my thickness      0             0   
645  felt tired but motivated to fly by this weeken...      0             0   
646  am gona do it all over again tomorrow is just ...      0             1   
647  monday to my beloved baby! am still here in th...      0             0   
648  capricorn moon with gemini mars its the part m...      0             0   

     Disgust  Fear  Joy  Love  Optimism  Pessimism 

In [ ]:
#save the file
df.to_csv('Emotional_English_dataset23_opt_label.csv', index=False)

In [ ]:
def get_url_patern():
 return re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0--9][a-za-z0--9-]+[a-zA-Z0--9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
 r'[a-zA-Z0--9]\.[^\s]{2,}|www\.[a-zA-Z0--9]\.[^\s]{2,})')

def get_hashtags_pattern():
 return re.compile(r'#\w\*')

def get_single_letter_words_pattern():
 return re.compile(r'(?<![\w-])\w(?![\w-])')

def get_blank_spaces_pattern():
 return re.compile(r'\s{2,}|\t')

def get_twitter_reserved_words_pattern():
 return re.compile(r'(RT|rt|FAV|fav|VIA|via)')

def get_mentions_pattern():
 return re.compile(r'@\w\*')

In [ ]:
def process*tweet(word):
word=re.sub(pattern=get_url_patern(), repl="", string=word)
word=re.sub(pattern=get_mentions_pattern(), repl="", string=word)
word=re.sub(pattern=get_hashtags_pattern(), repl="", string=word)
word=re.sub(pattern=get_twitter_reserved_words_pattern(), repl='', string=word)
word=re.sub (r'http\S+', "", word) # remove http links
word=re.sub(r'bit.ly/\S+', "", word) # remove bitly links
word=word.strip('[link]') # remove [links]
word=re.sub('(RT\s@[A-Za-z]+[A-Za-z0--9-*]+)', "", word) # remove retweet
word=re.sub('(@[A-Za-z]+[A-Za-z0--9-_]+)', "", word) # remove tweeted at
word=word.encode('ascii', 'ignore').decode('ascii')
return word